# Log

> Log functions and classes

In [ ]:
#| default_exp log

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.schemas import Query, Batch

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export

class Log():
    def __init__(self):
        self.batch_log = []
        
    def get_item(self, 
                 iteration: int, 
                 query_index: Optional[int]=None, 
                 result_index: Optional[int]=None):
        
        result = self.batch_log[iteration]
        
        if query_index is not None:
            result = result.queries[query_index]
            
        if result_index is not None:
            result = result.query_results[result_index]
        
        return result
    
    def add_batch(self, batch: Batch):
        iteration = len(self.batch_log)
        self.batch_log.append(batch)

    def compile_results(self, skip_removed: bool=True):
        output_dict = {}
        for batch in self.batch_log:
            for _, result in batch.enumerate_query_results(skip_removed):
                result_dict = result.model_dump()
                if result_dict.get('internal', None) is not None:
                    result_dict.pop('internal')
                    
                output_dict[result.id] = result_dict
                
        results = sorted([i for i in output_dict.values()], 
                         key=lambda x: x['score'] if x['score'] else float('-inf'), reverse=True)
        return results

In [ ]:
#| export

class Node():
    def __init__(self, query: Query, iteration: int):
        self.query = query
        self.iteration = iteration
        self.id = self.query.id
        self.parent = None
        self.children = {}
        self.prepare()
        
    def prepare(self):
        self.mean_score = None
        self.max_score = None
        self.removed = self.query.internal.removed
        
        result_scores = [i.score for i in self.query.valid_results() if (i.score is not None)]
        if result_scores:
            result_scores = np.array(result_scores)
            self.mean_score = result_scores.mean()
            self.max_score = result_scores.max()
            
    def add_child(self, child):
        child.parent = self
        self.children[child.id] = child
        
    def add_parent(self, parent):
        self.parent = parent
        parent.children[self.id] = self

In [ ]:
#| export

class QueryTree():
    def __init__(self):
        self.id_dict = {}
        self.nodes = []
        self.leaf_nodes = []
        self.final_nodes = []
        
    def build_tree(self, log):
        for iteration, batch in enumerate(log.batch_log):
            for query in batch:
                node = Node(query, iteration)
                self.nodes.append(node)
                self.id_dict[node.id] = node
                parent_node = self.id_dict.get(node.query.internal.parent_id, None)
                if parent_node is not None:
                    parent_node.add_child(node)
                    
        for node in self.nodes:
            if not node.children:
                self.leaf_nodes.append(node)
                
                if node.iteration == iteration:
                    self.final_nodes.append(node)